In [1]:
import pandas as pd
import numpy as np
from joblib import dump, load
from os import path
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [2]:
time_precision = 4   # jagame päeva xh suurusteks plokkideks


X_train = pd.read_csv("../../data/X_train.csv")
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.ToimAeg = X_train.ToimAeg.values.astype('datetime64[D]')
X_train.ToimAeg = X_train['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_train["Aasta"] = X_train['ToimAeg'].dt.year
X_train["Kuu"] = X_train['ToimAeg'].dt.month
X_train["Päev"] = X_train['ToimAeg'].dt.day
X_train["Tund"] = X_train['ToimAeg'].dt.hour
X_train.drop(columns="ToimAeg", inplace=True)
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.head()

y_train = pd.read_csv("../../data/y_train.csv")
y_train = y_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_train.head()

X_valid = pd.read_csv("../../data/X_valid.csv")
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.ToimAeg = X_valid.ToimAeg.values.astype('datetime64[D]')
X_valid.ToimAeg = X_valid['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_valid["Aasta"] = X_valid['ToimAeg'].dt.year
X_valid["Kuu"] = X_valid['ToimAeg'].dt.month
X_valid["Päev"] = X_valid['ToimAeg'].dt.day
X_valid["Tund"] = X_valid['ToimAeg'].dt.hour
X_valid.drop(columns="ToimAeg", inplace=True)
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.head()

y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.head()

X_train.head()

y_train.info()
y_train.drop(columns=["Kahjusumma","Koht"], inplace=True)
y_valid.drop(columns=["Kahjusumma", "Koht"], inplace=True)
y_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62476 entries, 0 to 62475
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LIIK_AVALIKU_KORRA_RIKKUMINE   62476 non-null  int8  
 1   LIIK_JALGRATTA_MOPEEDI_VARGUS  62476 non-null  int8  
 2   LIIK_KEHALINE_VAARKOHTLEMINE   62476 non-null  int8  
 3   LIIK_KELMUS                    62476 non-null  int8  
 4   LIIK_KONFLIKT/OMAVOLI          62476 non-null  int8  
 5   LIIK_KOOLIVAGIVALD             62476 non-null  int8  
 6   LIIK_LIIKLUSRIKKUMINE          62476 non-null  int8  
 7   LIIK_MOOTORSOIDUKI_VARGUS      62476 non-null  int8  
 8   LIIK_MUU                       62476 non-null  int8  
 9   LIIK_PISIVARGUS                62476 non-null  int8  
 10  LIIK_ROOVIMINE                 62476 non-null  int8  
 11  LIIK_TULEKAHJU                 62476 non-null  int8  
 12  LIIK_VANDALISM                 62476 non-null  int8  
 13  L

,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_JALGRATTA_MOPEEDI_VARGUS,LIIK_KEHALINE_VAARKOHTLEMINE,LIIK_KELMUS,LIIK_KONFLIKT/OMAVOLI,LIIK_KOOLIVAGIVALD,LIIK_LIIKLUSRIKKUMINE,LIIK_MOOTORSOIDUKI_VARGUS,LIIK_MUU,LIIK_PISIVARGUS,LIIK_ROOVIMINE,LIIK_TULEKAHJU,LIIK_VANDALISM,LIIK_VARGUS
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [4]:
knn = KNeighborsClassifier(n_neighbors=14).fit(X_train, y_train)

In [10]:
probs_knn = knn.predict(X_valid)

c:\python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [13]:
df_proba = pd.DataFrame(probs_knn, columns=y_valid.columns)
df_proba
#df_proba = pd.DataFrame(np.array([x.transpose()[1] for x in probs_knn]).transpose(), columns=y_valid.columns)

,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_JALGRATTA_MOPEEDI_VARGUS,LIIK_KEHALINE_VAARKOHTLEMINE,LIIK_KELMUS,LIIK_KONFLIKT/OMAVOLI,LIIK_KOOLIVAGIVALD,LIIK_LIIKLUSRIKKUMINE,LIIK_MOOTORSOIDUKI_VARGUS,LIIK_MUU,LIIK_PISIVARGUS,LIIK_ROOVIMINE,LIIK_TULEKAHJU,LIIK_VANDALISM,LIIK_VARGUS
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7804,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7805,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7806,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7807,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [14]:
for col in df_proba.columns:
    print(metrics.roc_auc_score(y_valid[col], df_proba[col]), col, sep="\t")

0.7013013256410914	LIIK_AVALIKU_KORRA_RIKKUMINE
0.5	LIIK_JALGRATTA_MOPEEDI_VARGUS
0.5091889334935693	LIIK_KEHALINE_VAARKOHTLEMINE
0.5	LIIK_KELMUS
0.5	LIIK_KONFLIKT/OMAVOLI
0.5	LIIK_KOOLIVAGIVALD
0.5	LIIK_LIIKLUSRIKKUMINE
0.5	LIIK_MOOTORSOIDUKI_VARGUS
0.5015851989589247	LIIK_MUU
0.671672361127569	LIIK_PISIVARGUS
0.5	LIIK_ROOVIMINE
0.5	LIIK_TULEKAHJU
0.5006251909477716	LIIK_VANDALISM
0.6375257577908249	LIIK_VARGUS


In [15]:
dump(knn, "trained_models/knn.joblib", compress=3, protocol=5)

['trained_models/knn.joblib']

Probably missused the knn thing, but since it doesnt seem like a good model for this anyway it does not really matter